# Imports

In [1]:
import os, time

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
from plotly import express as px

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

torch.__version__

'2.5.1'

setups

In [2]:
torch.manual_seed(44)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available, so using", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available, so using CPU only.")

CUDA is available, so using NVIDIA GeForce GTX 1660 Ti


# Data Loading

In [11]:
df = pd.read_csv("Data/QnA/100-unique-qna-dataset.csv")
df.columns = ['Question', 'Answer']
df

,Question,Answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100
...,...,...
85,Who directed the movie 'Titanic'?,JamesCameron
86,Which superhero is also known as the Dark Knight?,Batman
87,What is the capital of Brazil?,Brasilia
88,Which fruit is known as the king of fruits?,Mango


tokenize

In [12]:
def tokenize(text):
    text = text.strip() # removing whitespaces
    text = text.lower() # lowercase words
    text = text.replace('?', '').replace("'", '') # removing ?, '
    text = text.split(' ') # creating tokens
    text = [word.strip() for word in text] # removing word whitespaces
    
    return text

text = "What is the capital of 'France'?"
tokenize(text)

['what', 'is', 'the', 'capital', 'of', 'france']

vocab space

In [ ]:
vocab_space = {'<UNK>': 0} # for out of vocab

def build_vocab_space(row):
    
    que_tokens = tokenize(row['Question'])
    ans_tokens = tokenize(row['Answer'])
    tokens = que_tokens + ans_tokens

    for token in tokens:
        if not vocab_space.get(token):
            vocab_space[token] = len(vocab_space)


_ = df.apply(build_vocab_space, axis=1)

len(vocab_space)

324

vectors

In [ ]:
def text_to_vector(text, vocab_space):

    vector = []
    for token in tokenize(text):
        if vocab_space.get(token):
            val = vocab_space[token]
        else:
            val = vocab_space['<UNK>']
        
        vector.append(val)

    return vector


text = "Ahbaz What is the capital of 'France'?"
text_to_vector(text, vocab_space)

[0, 1, 2, 3, 4, 5, 6]